In [1]:
%reload_ext autoreload
%autoreload 2

# Imports
Import the required classes and functions.

In [2]:
from mleko.dataset.ingest import KaggleIngester
from mleko.dataset.convert import CSVToVaexConverter
from mleko.dataset.split import RandomSplitter, ExpressionSplitter
from mleko.pipeline import Pipeline
from mleko.pipeline.steps import IngestStep, ConvertStep, SplitStep, FeatureSelectStep
from mleko.dataset.feature_select import MissingRateFeatureSelector, StandardDeviationFeatureSelector

# Constants
Define configuration variables.

In [3]:
OWNER_SLUG = 'mlg-ulb'
DATASET_SLUG = 'creditcardfraud'
DATASET_NAME = f'{OWNER_SLUG}/{DATASET_SLUG}'

TARGET_FEATURE = "Class"
TIME_FEATURE = "Time"
META_FEATURES = [TIME_FEATURE, TARGET_FEATURE]
RANDOM_STATE = 1337

# Pipeline Setup

In [57]:
kaggle_data_source = KaggleIngester(f"data/{DATASET_NAME}/raw", owner_slug=OWNER_SLUG, dataset_slug=DATASET_SLUG)
csv_to_arrow_converter = CSVToVaexConverter(
    output_directory=f"data/{DATASET_NAME}/converted", downcast_float=True, random_state=RANDOM_STATE
)
random_data_splitter = RandomSplitter(
    output_directory=f"data/{DATASET_NAME}/split",
    data_split=(0.80, 0.20),
    shuffle=True,
    stratify=TARGET_FEATURE,
    random_state=RANDOM_STATE,
)
expression_data_splitter = ExpressionSplitter(output_directory=f"data/{DATASET_NAME}/split", expression="Time > 100")
missing_rate_feature_selector = MissingRateFeatureSelector(
    output_directory=f"data/{DATASET_NAME}/feature_selection", missing_rate_threshold=0.3, ignore_features=META_FEATURES
)
standard_deviation_feature_selector = StandardDeviationFeatureSelector(
    output_directory=f"data/{DATASET_NAME}/feature_selection",
    standard_deviation_threshold=0.00,
    ignore_features=META_FEATURES,
)


pipeline = Pipeline(
    steps=[
        IngestStep(kaggle_data_source, outputs=["raw_csv"]),
        ConvertStep(csv_to_arrow_converter, inputs=["raw_csv"], outputs=["df_clean"]),
        SplitStep(random_data_splitter, inputs=["df_clean"], outputs=["df_train_validate", "df_test"]),
        SplitStep(expression_data_splitter, inputs=["df_train_validate"], outputs=["df_train", "df_validate"]),
        FeatureSelectStep(missing_rate_feature_selector, inputs=["df_train"], outputs=["df_train_missing_rate"]),
        FeatureSelectStep(
            standard_deviation_feature_selector,
            inputs=["df_train_missing_rate"],
            outputs=["df_train_standard_deviation"],
        ),
    ]
)


[2023-05-25 22:39:45] [INFO] Attempting to fetch Kaggle API credentials from environment variables 'KAGGLE_USERNAME' and 'KAGGLE_KEY'. (kaggle_ingester.py:77)
[2023-05-25 22:39:45] [WARNING] Kaggle API credentials not found in environment variables, attempting to fetch from fallback path at ~/.kaggle/kaggle.json. (kaggle_ingester.py:85)
[2023-05-25 22:39:45] [INFO] Kaggle credentials successfully fetched. (kaggle_ingester.py:94)


# Run Pipeline

In [59]:
data_container =  pipeline.run().data

[2023-05-25 22:39:58] [INFO] No data container provided. Creating an empty one. (pipeline.py:77)
[2023-05-25 22:39:58] [INFO] Executing step 1: IngestStep. (pipeline.py:81)
[2023-05-25 22:39:59] [INFO] Cache Hit: Local dataset is up to date with Kaggle, skipping download. (kaggle_ingester.py:283)
[2023-05-25 22:39:59] [INFO] Finished step 1 execution. (pipeline.py:83)
[2023-05-25 22:39:59] [INFO] Executing step 2: ConvertStep. (pipeline.py:81)
[2023-05-25 22:39:59] [INFO] Cache Hit (LRUCache) CSVToVaexConverter.convert: Using cached output. (cache_mixin.py:126)
[2023-05-25 22:39:59] [INFO] Finished step 2 execution. (pipeline.py:83)
[2023-05-25 22:39:59] [INFO] Executing step 3: SplitStep. (pipeline.py:81)
[2023-05-25 22:39:59] [INFO] Cache Hit (LRUCache) RandomSplitter.split: Using cached output. (cache_mixin.py:126)
[2023-05-25 22:39:59] [INFO] Finished step 3 execution. (pipeline.py:83)
[2023-05-25 22:39:59] [INFO] Executing step 4: SplitStep. (pipeline.py:81)
[2023-05-25 22:39:59] 